<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/PerceiverIO_lucidrains_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor #, Lambda, Compose
import matplotlib.pyplot as plt
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

training_data = datasets.FashionMNIST(root="data", train=True, download=True,transform=ToTensor(),)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor(),)
# print(training_data)
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# test_dataloader: #image, label
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
!pip install perceiver-pytorch


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Using cuda device


In [3]:
# # https://github.com/lucidrains/perceiver-pytorch
# import torch
# from perceiver_pytorch import PerceiverIO

# model = PerceiverIO(
#     dim = 28,                    # dimension of sequence to be encoded
#     queries_dim = 32,            # Q dimension of decoder queries
#     logits_dim = 10,            # dimension of final logits
#     depth = 6,                   # depth of net
#     num_latents = 256,           # N number of latents, or induced set points, or centroids. different papers giving it different names
#     latent_dim = 512,            # D latent dimension
#     cross_heads = 1,             # number of heads for cross attention. paper said 1
#     latent_heads = 8,            # number of heads for latent self attention, 8
#     cross_dim_head = 64,         # number of dimensions per cross attention head
#     latent_dim_head = 64,        # number of dimensions per latent self attention head
#     weight_tie_layers = False    # whether to weight tie layers (optional, as indicated in the diagram)
#     ).to(device)

# seq = torch.randn(1, 28, 28, device=device) #(batch, latent_dim, dim)
# queries = torch.randn(1, 32, device=device) #(decoder seq, queries_dim)

# logits = model(seq, queries = queries) # (batch, decoder seq, logits dim) range (-1,1)
# print(logits.shape) #torch.Size(batch, decoder seq, logits dim)
# logits=logits.squeeze()




In [15]:
# https://github.com/lucidrains/perceiver-pytorch
import torch
from perceiver_pytorch import PerceiverIO

model = PerceiverIO(
    dim = 28*28,                 # C dimension of sequence to be encoded
    queries_dim = 32,            # Q dimension of decoder queries
    logits_dim = 10,            # E dimension of final logits
    depth = 6,                   # depth of net
    num_latents = 256,           # N number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 512,            # D latent dimension
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    latent_heads = 8,            # number of heads for latent self attention, 8
    cross_dim_head = 64,         # number of dimensions per cross attention head
    latent_dim_head = 64,        # number of dimensions per latent self attention head
    weight_tie_layers = False    # whether to weight tie layers (optional, as indicated in the diagram)
    ).to(device)

seq = torch.randn(1, 14, 28*28, device=device) #(batch, latent_dim?, dim)
# seq = torch.randn(25,13,28*28, device=device) #(batch, latent_dim, dim)

queries = torch.ones(5, 32, device=device) #(decoder seq, queries_dim)

# logits = model(seq, queries = None) # (batch, decoder seq, logits dim) range (-1,1)
logits = model(seq, queries = queries) # (batch, decoder seq, logits dim) range (-1,1)

# print(logits)
print(logits.shape) #torch.Size(batch, decoder seq, logits dim)
logits=logits.squeeze()
print(logits.shape)


torch.Size([1, 5, 32])
torch.Size([5, 32])


In [ ]:
X = torch.rand(1, 28, 28, device=device)
X=X.flatten()
X=X.view(1,1,28*28)

print(X.shape)
logits = model(X,queries=queries)
logits=logits.squeeze()
print(logits)
# pred_probab = nn.Softmax(dim=0)(logits)
# print(pred_probab)
# y_pred = pred_probab.argmax(0)
y_pred = logits.argmax(0)
print(f"Predicted class: {y_pred}")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # pred = model(X) # Compute prediction error
        X=X.squeeze()
        pred = model(X,queries=queries)
        pred=pred.squeeze()
        loss = loss_fn(pred, y)
        optimizer.zero_grad() # Backpropagation
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            X=X.squeeze()
            pred = model(X,queries=queries)
            pred=pred.squeeze()
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [ ]:
test(test_dataloader, model, loss_fn)


In [ ]:

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model.pth"))


In [ ]:
classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot",]
model.eval()
import random
n=random.randint(0,1000)
print(n)
x, y = test_data[n][0], test_data[n][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
